In [1]:
import pybamm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [2]:
parameter_values = pybamm.ParameterValues(chemistry = pybamm.parameter_sets.Chen2020)

In [3]:
model = pybamm.lithium_ion.DFN()

In [4]:
#Base Line Temperatures at 298.15

sim = pybamm.Simulation(model)
sim.solve([0,3600])

In [5]:
# Pivot value for sensitive analysis

t_sens = sim.solution["Time [s]"]
V_sens = sim.solution["Terminal voltage [V]"]

V_sens.entries


array([3.7716126 , 3.75501643, 3.74799998, 3.74283657, 3.73829449,
       3.73402476, 3.7299169 , 3.72593365, 3.72206182, 3.71829739,
       3.71463942, 3.7110873 , 3.70763967, 3.70429335, 3.70104273,
       3.6978796 , 3.69479459, 3.69178036, 3.68883334, 3.68595342,
       3.6831414 , 3.68039825, 3.67772448, 3.67511987, 3.67258356,
       3.67011426, 3.66770968, 3.66536779, 3.66308569, 3.66086015,
       3.65868771, 3.6565645 , 3.65448577, 3.65244688, 3.65044205,
       3.64846433, 3.64650549, 3.64455505, 3.64259867, 3.64061648,
       3.63857832, 3.63643753, 3.63412433, 3.63155799, 3.62870437,
       3.62563468, 3.62250396, 3.61947978, 3.61668815, 3.6141904 ,
       3.61198854, 3.61004748, 3.60831933, 3.60675911, 3.60533076,
       3.60400827, 3.60277338, 3.60161395, 3.60052223, 3.59949294,
       3.59852078, 3.59759705, 3.59670572, 3.59582424, 3.59493114,
       3.59401287, 3.5930631 , 3.59208113, 3.59106658, 3.59001999,
       3.58894145, 3.58783056, 3.58668577, 3.58550502, 3.58428

In [6]:
parameter_values.search("temperature")

Ambient temperature [K]	298.15
Initial temperature [K]	298.15
Reference temperature [K]	298.15


In [7]:
#Setting up Array for looping temperature data for Chen2020 
Start = 100
Stop = 500
Step = 500
Temperature = np.linspace(Start,Stop,num=Step)
print(Temperature)

[100.         100.80160321 101.60320641 102.40480962 103.20641283
 104.00801603 104.80961924 105.61122244 106.41282565 107.21442886
 108.01603206 108.81763527 109.61923848 110.42084168 111.22244489
 112.0240481  112.8256513  113.62725451 114.42885772 115.23046092
 116.03206413 116.83366733 117.63527054 118.43687375 119.23847695
 120.04008016 120.84168337 121.64328657 122.44488978 123.24649299
 124.04809619 124.8496994  125.65130261 126.45290581 127.25450902
 128.05611222 128.85771543 129.65931864 130.46092184 131.26252505
 132.06412826 132.86573146 133.66733467 134.46893788 135.27054108
 136.07214429 136.87374749 137.6753507  138.47695391 139.27855711
 140.08016032 140.88176353 141.68336673 142.48496994 143.28657315
 144.08817635 144.88977956 145.69138277 146.49298597 147.29458918
 148.09619238 148.89779559 149.6993988  150.501002   151.30260521
 152.10420842 152.90581162 153.70741483 154.50901804 155.31062124
 156.11222445 156.91382766 157.71543086 158.51703407 159.31863727
 160.12024

In [8]:
result = []

for i in Temperature:
    
    parameter_values["Ambient temperature [K]"] = i
    
    parameter_values["Initial temperature [K]"] = i
    
    parameter_values["Reference temperature [K]"] = i
    
    param = model.default_parameter_values
    
    # Load Solver
    fast_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="fast")
    
    # Create Sim
    fast_sim = pybamm.Simulation(model, parameter_values=param, solver=fast_solver)
    
    # Solve
    fast_sim.solve([0,3600])
    print("Fast mode solve time: {}".format(fast_sim.solution.solve_time))
    
    solution = fast_sim.solution
    
    t = solution["Time [s]"]
    V = solution["Terminal voltage [V]"]
    
    V_SA = ((V_sens.entries - V.entries)/V_sens.entries)*100
    
    V_SA_min = np.amin(V_SA)
    
    V_SA_max = np.amax(V_SA)
    
    
    print("For Temperature: {}".format(i),"K , Min Sensitivity is: {}".format(V_SA_min),"% and Max Sensitivity is: {}".format(V_SA_max))
    
    result.append((i,V_SA_min,V_SA_max))
    
    np.savetxt('Voltage_Sens.csv',(i,V_SA_min,V_SA_max))
    

Fast mode solve time: 0.12016440002480522
For Temperature: 100.0 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.12432230002013966
For Temperature: 100.80160320641282 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.11498489999212325
For Temperature: 101.60320641282566 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.11118829995393753
For Temperature: 102.40480961923848 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.16348679998191074
For Temperature: 103.2064128256513 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.12460269994335249
For Temperature: 104.00801603206413 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.0059168

Fast mode solve time: 0.14747129997704178
For Temperature: 139.27855711422845 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.13749100000131875
For Temperature: 140.08016032064128 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.11280290002468973
For Temperature: 140.88176352705412 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.16743099997984245
For Temperature: 141.68336673346693 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.14741590002086014
For Temperature: 142.48496993987976 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.1431954000145197
For Temperature: 143.2865731462926 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity i

Fast mode solve time: 0.10948909999569878
For Temperature: 178.5571142284569 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.10242959996685386
For Temperature: 179.35871743486973 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.10646120004821569
For Temperature: 180.16032064128257 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.11771710001630709
For Temperature: 180.9619238476954 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.12342630000784993
For Temperature: 181.76352705410824 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.11443529999814928
For Temperature: 182.56513026052104 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity i

Fast mode solve time: 0.1347230999963358
For Temperature: 217.83567134268537 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.6090557999559678
For Temperature: 218.6372745490982 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.28338580002309754
For Temperature: 219.43887775551104 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.26283979997970164
For Temperature: 220.24048096192385 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.1564695000415668
For Temperature: 221.04208416833666 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.17608290002681315
For Temperature: 221.8436873747495 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 

Fast mode solve time: 0.10787579999305308
For Temperature: 257.1142284569138 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.10438249999424443
For Temperature: 257.9158316633267 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.1433316000038758
For Temperature: 258.71743486973946 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.1149301000405103
For Temperature: 259.5190380761523 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.11926270002732053
For Temperature: 260.32064128256513 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.10733600001549348
For Temperature: 261.12224448897797 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 

Fast mode solve time: 0.11273970000911504
For Temperature: 296.3927855711423 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.13815880002221093
For Temperature: 297.1943887775551 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.15832260000752285
For Temperature: 297.99599198396794 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.11613480001688004
For Temperature: 298.7975951903808 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.12030389998108149
For Temperature: 299.59919839679355 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.10717650002334267
For Temperature: 300.40080160320645 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is

Fast mode solve time: 0.14229940000222996
For Temperature: 335.67134268537075 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.1509061999968253
For Temperature: 336.4729458917836 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.1451498999958858
For Temperature: 337.2745490981964 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.12137840001378208
For Temperature: 338.0761523046092 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.10786390001885593
For Temperature: 338.8777555110221 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0.005916809059446171
Fast mode solve time: 0.12246370001230389
For Temperature: 339.67935871743487 K , Min Sensitivity is: -0.026741916142297262 % and Max Sensitivity is: 0

KeyboardInterrupt: 